In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print(F'Список колонок в train.csv ({len(list(train.columns.values))}): {list(train.columns.values)}\n')
print(F'Список колонок в test.csv ({len(list(test.columns.values))}): {list(test.columns.values)}')

train.head()

Список колонок в train.csv (21): ['id', 'Leave_Next_Month', 'Gender', 'Pensioner', 'Married', 'Children', 'Months_with_company', 'Phone_Service', 'Multiple_Lines', 'Internet_Service', 'Online_Security', 'Online_Backup', 'Device_Protection', 'Tech_Support', 'Streaming_TV', 'Streaming_Movies', 'Type_contract', 'Paperless_Billing', 'Payment_Method', 'Monthly_Payment', 'Total_Payment']

Список колонок в test.csv (20): ['id', 'Gender', 'Pensioner', 'Married', 'Children', 'Months_with_company', 'Phone_Service', 'Multiple_Lines', 'Internet_Service', 'Online_Security', 'Online_Backup', 'Device_Protection', 'Tech_Support', 'Streaming_TV', 'Streaming_Movies', 'Type_contract', 'Paperless_Billing', 'Payment_Method', 'Monthly_Payment', 'Total_Payment']


,id,Leave_Next_Month,Gender,Pensioner,Married,Children,Months_with_company,Phone_Service,Multiple_Lines,Internet_Service,...,Online_Backup,Device_Protection,Tech_Support,Streaming_TV,Streaming_Movies,Type_contract,Paperless_Billing,Payment_Method,Monthly_Payment,Total_Payment
0,4,1,F,0,N,N,2,Y,N,Fiber optic,...,N,N,N,N,N,Per_month,Y,Check_electron,70.70,151.65
1,7,0,F,0,N,N,10,N,No_phone,DSL,...,N,N,N,N,N,Per_month,N,Check_mailed,29.75,301.9
2,8,1,F,0,Y,N,28,Y,Y,Fiber optic,...,N,Y,Y,Y,Y,Per_month,Y,Check_electron,104.80,3046.05
3,9,0,M,0,N,Y,62,Y,N,DSL,...,Y,N,N,N,N,Year-1,N,Bank_transfer_auto,56.15,3487.95
4,11,0,M,0,N,N,16,Y,N,N,...,No_internet,No_internet,No_internet,No_internet,No_internet,Year-2,N,Credit_card_auto,18.95,326.8


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
features = ['Gender', 'Pensioner', 'Married', 'Children', 'Months_with_company', 'Phone_Service', 'Multiple_Lines', 
            'Internet_Service', 'Online_Security', 'Online_Backup', 'Device_Protection', 'Tech_Support', 'Streaming_TV', 
            'Streaming_Movies', 'Type_contract', 'Paperless_Billing', 'Payment_Method', 'Monthly_Payment', 'Total_Payment']

In [3]:
train.dtypes

id                       int64
Leave_Next_Month         int64
Gender                  object
Pensioner                int64
Married                 object
Children                object
Months_with_company      int64
Phone_Service           object
Multiple_Lines          object
Internet_Service        object
Online_Security         object
Online_Backup           object
Device_Protection       object
Tech_Support            object
Streaming_TV            object
Streaming_Movies        object
Type_contract           object
Paperless_Billing       object
Payment_Method          object
Monthly_Payment        float64
Total_Payment           object
dtype: object

In [4]:
train.loc[ train['Total_Payment'] == ' ', 'Total_Payment'] = '0'

In [5]:
test.loc[ test['Total_Payment'] == ' ', 'Total_Payment'] = '0'

In [6]:
train[train['Total_Payment'] =='0']

,id,Leave_Next_Month,Gender,Pensioner,Married,Children,Months_with_company,Phone_Service,Multiple_Lines,Internet_Service,...,Online_Backup,Device_Protection,Tech_Support,Streaming_TV,Streaming_Movies,Type_contract,Paperless_Billing,Payment_Method,Monthly_Payment,Total_Payment
375,753,0,M,0,N,Y,0,Y,N,N,...,No_internet,No_internet,No_internet,No_internet,No_internet,Year-2,N,Check_mailed,20.25,0
691,1340,0,F,0,Y,Y,0,N,No_phone,DSL,...,Y,Y,Y,Y,N,Year-2,N,Credit_card_auto,56.05,0
1717,3331,0,M,0,Y,Y,0,Y,N,N,...,No_internet,No_internet,No_internet,No_internet,No_internet,Year-2,N,Check_mailed,19.85,0
1962,3826,0,M,0,Y,Y,0,Y,Y,N,...,No_internet,No_internet,No_internet,No_internet,No_internet,Year-2,N,Check_mailed,25.35,0


In [7]:
sorted(train['Total_Payment'].unique())

['0',
 '100.2',
 '100.25',
 '100.4',
 '100.9',
 '1001.5',
 '1004.35',
 '1004.5',
 '1006.9',
 '1007.8',
 '1008.55',
 '1008.7',
 '1009.25',
 '101.1',
 '1010',
 '1011.5',
 '1011.8',
 '1012.4',
 '1017.35',
 '102.45',
 '102.5',
 '102.75',
 '1020.2',
 '1020.75',
 '1022.5',
 '1022.95',
 '1023.9',
 '1024.7',
 '1025.05',
 '1025.15',
 '1026.35',
 '1028.75',
 '1029.35',
 '1031.1',
 '1031.4',
 '1033',
 '1033.9',
 '1033.95',
 '1035.5',
 '1039.45',
 '104.3',
 '1041.8',
 '1042.65',
 '1043.4',
 '1045.25',
 '1046.1',
 '1046.2',
 '1046.5',
 '1047.7',
 '1048.45',
 '1049.05',
 '1050.5',
 '1051.05',
 '1052.4',
 '1054.75',
 '1054.8',
 '1057',
 '1057.55',
 '1057.85',
 '1058.1',
 '1058.25',
 '106.2',
 '106.85',
 '106.9',
 '1062.1',
 '1067.65',
 '1068.15',
 '1068.85',
 '107.05',
 '107.1',
 '1070.15',
 '1071.4',
 '1071.6',
 '1072',
 '1074.3',
 '1074.65',
 '1077.05',
 '1077.5',
 '108.65',
 '108.7',
 '1080.55',
 '1081.25',
 '1081.45',
 '1082.8',
 '1083',
 '1087.45',
 '1088.25',
 '109.2',
 '109.8',
 '1092.35',
 '1

In [8]:
train['Total_Payment'] = train['Total_Payment'].astype('float64')

In [9]:
test['Total_Payment'] = test['Total_Payment'].astype('float64')

In [10]:
params_xgb = {
    "objective": "binary:logistic",
    "eval_metric": 'logloss',
    "eta": 0.05,
    "max_depth": 2,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "verbosity": 0,
    "silent": True,
}
num_boost_round = 500
early_stopping_rounds = 10

In [11]:
X = train.drop('Leave_Next_Month', axis=1)
X = X.drop('id', axis=1)
X = X.drop('Monthly_Payment', axis=1)
X = X.drop('Total_Payment', axis=1)
y = train['Leave_Next_Month']


label_encoders = []
X_encoded = pd.DataFrame()
for column in X.columns:
    label_encoder = LabelEncoder()
    X_encoded[column] = label_encoder.fit_transform(X[column])
    label_encoders.append(label_encoder)

X_encoded["Monthly_Payment"] = train['Monthly_Payment']
X_encoded["Total_Payment"] = train['Total_Payment']    

X_test = test

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

label_encoders = []
test_to_encode = test.drop("id", axis=1)
test_to_encode = test_to_encode.drop('Monthly_Payment', axis=1)
test_to_encode = test_to_encode.drop('Total_Payment', axis=1)

X_encoded_test = pd.DataFrame()
for column in test_to_encode.columns:
    label_encoder = LabelEncoder()
    X_encoded_test[column] = label_encoder.fit_transform(test_to_encode[column])
    label_encoders.append(label_encoder)

X_encoded_test["Monthly_Payment"] = test['Monthly_Payment']
X_encoded_test["Total_Payment"] = test['Total_Payment']    

In [12]:
dtrain = xgb.DMatrix(X_train.values, y_train.values)
dvalid = xgb.DMatrix(X_test.values, y_test.values)
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
gbm = xgb.train(params_xgb, dtrain, num_boost_round, evals=watchlist,
                early_stopping_rounds=early_stopping_rounds, verbose_eval=10)

[0]	train-logloss:0.57605	eval-logloss:0.54953
[10]	train-logloss:0.50725	eval-logloss:0.48300
[20]	train-logloss:0.47103	eval-logloss:0.44801
[30]	train-logloss:0.45112	eval-logloss:0.42900
[40]	train-logloss:0.43729	eval-logloss:0.41716
[50]	train-logloss:0.42690	eval-logloss:0.40677
[60]	train-logloss:0.41978	eval-logloss:0.40060
[70]	train-logloss:0.41402	eval-logloss:0.39508
[80]	train-logloss:0.40967	eval-logloss:0.39326
[90]	train-logloss:0.40622	eval-logloss:0.39037
[100]	train-logloss:0.40313	eval-logloss:0.38750
[110]	train-logloss:0.40029	eval-logloss:0.38623
[120]	train-logloss:0.39822	eval-logloss:0.38544
[130]	train-logloss:0.39634	eval-logloss:0.38374
[140]	train-logloss:0.39468	eval-logloss:0.38324
[150]	train-logloss:0.39291	eval-logloss:0.38289
[155]	train-logloss:0.39212	eval-logloss:0.38327


In [13]:
pred = gbm.predict(xgb.DMatrix(X_encoded_test.values))

In [14]:
sample = pd.read_csv('sample_submission.csv')
print(sample.axes[0])
print(sample.axes[1])
sample

RangeIndex(start=0, stop=3522, step=1)
Index(['id', 'Leave_Next_Month'], dtype='object')


,id,Leave_Next_Month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,5,0.5
...,...,...
3517,7032,0.5
3518,7034,0.5
3519,7037,0.5
3520,7039,0.5


In [15]:
import csv

with open("result.csv", "w", newline="") as f:

    writer = csv.writer(f)

    writer.writerow(["id", "Leave_Next_Month"])
    for i in range(len(pred)):    
        writer.writerow([test["id"][i], pred[i]]) 
result = pd.read_csv('result.csv')
result

,id,Leave_Next_Month
0,0,0.706786
1,1,0.062429
2,2,0.288643
3,3,0.045938
4,5,0.810573
...,...,...
3517,7032,0.872861
3518,7034,0.264855
3519,7037,0.009656
3520,7039,0.084899
